In [ ]:
# import modules & set up logging
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 

In [ ]:
class OffensiveSentences(object):
    def __init__(self):
        pass
 
    def __iter__(self):
        for line in open(os.path.join('./full_textOnly_cleaned_dataset.csv')):
            yield line.split()
 
sentences = OffensiveSentences() # a memory-friendly iterator

In [ ]:
model = Word2Vec(sentences, min_count=2, size=50, workers=4)

In [ ]:
model.save('w2v_5dim_model')
new_model = gensim.models.Word2Vec.load('w2v_5dim_model')

In [ ]:
model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)

In [ ]:
model.doesnt_match("breakfast cereal dinner lunch".split())

In [ ]:
model.similarity('woman', 'man')

In [ ]:
model['computer']

In [ ]:
model.wv.most_similar('computer', topn=10)